## Add 2017 movies to the previous dataset 

In [28]:
import pandas as pd
import numpy as np

In [29]:
credits = pd.read_csv("Data/credits.csv")

In [30]:
credits.head()

cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3  [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4  [{'cast_id': 1, 'character': 'George Banks', '...   

                                                crew     id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844  
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  31357  
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...  11862

In [31]:
# Renaming the 'id' column in the credits dataframe with 'movie_id'
credits.rename(columns= {'id': 'movie_id'}, inplace= True)

In [32]:
metadata = pd.read_csv("Data/movies_metadata.csv")

/Users/ashanth/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
metadata.head()

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview  ... release_date  \
0  Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2  A family wedding reignites the ancient feud be...  ...   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4  Just when George Banks has recovered from his ...  ...   1995-02-10   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   
3  Released  Friends are the people who let you be yourself...   
4  Released  Just When His World Is Back To Normal... He's ...   

                         title  video vote_average vote_count  
0                    Toy Story  False          7.7     5415.0  
1                      Jumanji  False          6.9     2413.0  
2             Grumpier Old Men  False          6.5       92.0  
3            Waiting to Exhale  False          6.1       34.0  
4  Father of the Bride Part II  False          5.7      173.0  

[5 rows x 24 columns]

In [34]:
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [35]:
# We want to get 2017 movies so we have to convert the release_date into a python date time object
metadata['release_date'] = pd.to_datetime(metadata['release_date'],  errors='coerce')

In [36]:
metadata['year'] = pd.DatetimeIndex(metadata['release_date']).year

In [37]:
# Renaming the 'id' column in the metadata dataframe with 'movie_id'
metadata.rename(columns= {'id': 'movie_id'}, inplace= True)

In [38]:
 # taking out the 2017 movies data
new_metadata = metadata.loc[metadata['year'] == 2017, ['genres', 'movie_id', 'original_title', 'overview']]

In [39]:
# We need to convert string type movie_id to integer
new_metadata['movie_id'] = new_metadata['movie_id'].astype(int)

In [40]:
# Merging the two dataset
movies_df = new_metadata.merge(credits, on= 'movie_id')

In [41]:
movies_df = movies_df.drop(columns= ['crew'])

In [42]:
# Converting the string of brackets and curly braces into lists of dictionary objects
from ast import literal_eval
elements = ['genres', 'cast']
for element in elements:
    movies_df[element] = movies_df[element].apply(literal_eval)

In [43]:
# Taking out the cast and genres names from the lists
def get_name(data):
    if isinstance(data, list):
        names = [ col['name'] for col in data ]
        if len(names) > 3:
            names = names[:3]
        return names
    return []  # returning empty list in case of missing data

In [44]:
for element in elements:
    movies_df[element] = movies_df[element].apply(get_name)

In [45]:
movies_df.head()

genres  movie_id  \
0  [Adventure, Action, Fantasy]    166426   
1  [Action, Adventure, Fantasy]    141052   
2  [Action, Adventure, Fantasy]    284053   
3   [Action, Adventure, Comedy]    283995   
4  [Fantasy, Action, Adventure]    245842   

                                     original_title  \
0  Pirates of the Caribbean: Dead Men Tell No Tales   
1                                    Justice League   
2                                    Thor: Ragnarok   
3                    Guardians of the Galaxy Vol. 2   
4                               The King's Daughter   

                                            overview  \
0  Thrust into an all-new paycheck, a down-on-his...   
1  Fueled by his restored faith in humanity and i...   
2  Thor is imprisoned on the other side of the un...   
3  The Guardians must fight to keep their newfoun...   
4  King Louis XIV's quest for immortality leads h...   

                                                cast  
0        [Johnny Depp, Javier Bardem, Geoffrey Rush]  
1             [Ben Affleck, Henry Cavill, Gal Gadot]  
2  [Chris Hemsworth, Tom Hiddleston, Cate Blanchett]  
3          [Chris Pratt, Zoe Saldana, Dave Bautista]  
4    [Pierce Brosnan, William Hurt, Benjamin Walker]

In [46]:
movies_df.isnull().sum()

genres             0
movie_id           0
original_title     0
overview          12
cast               0
dtype: int64

In [47]:
old_data = pd.read_csv("Data/data1.csv")

In [48]:
new_data = old_data.append(movies_df)

In [49]:
new_data.drop_duplicates(subset= 'original_title', keep= 'last', inplace= True)

In [50]:
new_data.shape

(5321, 5)

In [51]:
new_data.isnull().sum()

genres             0
movie_id           0
original_title     0
overview          12
cast               0
dtype: int64

In [52]:
new_data = new_data.dropna()

In [53]:
new_data.isnull().sum()

genres            0
movie_id          0
original_title    0
overview          0
cast              0
dtype: int64

In [55]:
new_data.to_csv("Data/data2.csv", index= False)